Deep Learning
=============

Assignment 4
------------

Previously in `2_fullyconnected.ipynb` and `3_regularization.ipynb`, we trained fully connected networks to classify [notMNIST](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html) characters.

The goal of this assignment is make the neural network convolutional.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28) (100000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a TensorFlow-friendly shape:
- convolutions need the image data formatted as a cube (width by height by #channels)
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10
num_channels = 1 # grayscale

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
valid_dataset, valid_labels = valid_dataset[:2000], valid_labels[:2000]
test_dataset, test_labels = reformat(test_dataset, test_labels)
test_dataset, test_labels = test_dataset[:2000], test_labels[:2000]
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (100000, 28, 28, 1) (100000, 10)
Validation set (2000, 28, 28, 1) (2000, 10)
Test set (2000, 28, 28, 1) (2000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

Let's build a small network with two convolutional layers, followed by one fully connected layer. Convolutional networks are more expensive computationally, so we'll limit its depth and number of fully connected nodes.

In [5]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer1_biases)
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 2, 2, 1], padding='SAME')
    hidden = tf.nn.relu(conv + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [13]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.494696
Minibatch accuracy: 12.5%
Validation accuracy: 10.9%
Minibatch loss at step 50: 1.127407
Minibatch accuracy: 81.2%
Validation accuracy: 44.7%
Minibatch loss at step 100: 1.178413
Minibatch accuracy: 56.2%
Validation accuracy: 72.9%
Minibatch loss at step 150: 0.483335
Minibatch accuracy: 81.2%
Validation accuracy: 79.0%
Minibatch loss at step 200: 0.483923
Minibatch accuracy: 87.5%
Validation accuracy: 77.0%
Minibatch loss at step 250: 0.941822
Minibatch accuracy: 75.0%
Validation accuracy: 76.8%
Minibatch loss at step 300: 0.118579
Minibatch accuracy: 100.0%
Validation accuracy: 80.8%
Minibatch loss at step 350: 0.347785
Minibatch accuracy: 93.8%
Validation accuracy: 80.5%
Minibatch loss at step 400: 0.611616
Minibatch accuracy: 81.2%
Validation accuracy: 81.7%
Minibatch loss at step 450: 0.519599
Minibatch accuracy: 81.2%
Validation accuracy: 81.8%
Minibatch loss at step 500: 1.083658
Minibatch accuracy: 75.0%
Validation accuracy: 79.9%


---
Problem 1
---------

The convolutional model above uses convolutions with stride 2 to reduce the dimensionality. Replace the strides by a max pooling operation (`nn.max_pool()`) of stride 2 and kernel size 2.

---

In [7]:
batch_size = 16
patch_size = 5
depth = 16
num_hidden = 64

graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  layer1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  layer1_biases = tf.Variable(tf.zeros([depth]))
  layer2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth], stddev=0.1))
  layer2_biases = tf.Variable(tf.constant(1.0, shape=[depth]))
  layer3_weights = tf.Variable(tf.truncated_normal(
      [image_size // 4 * image_size // 4 * depth, num_hidden], stddev=0.1))
  layer3_biases = tf.Variable(tf.constant(1.0, shape=[num_hidden]))
  layer4_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  layer4_biases = tf.Variable(tf.constant(1.0, shape=[num_labels]))
  
  # Model.
  def model(data):
    conv = tf.nn.conv2d(data, layer1_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer1_biases)  
    conv = tf.nn.conv2d(hidden, layer2_weights, [1, 1, 1, 1], padding='SAME')
    hidden = tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding='SAME')
    hidden = tf.nn.relu(hidden + layer2_biases)
    shape = hidden.get_shape().as_list()
    reshape = tf.reshape(hidden, [shape[0], shape[1] * shape[2] * shape[3]])
    hidden = tf.nn.relu(tf.matmul(reshape, layer3_weights) + layer3_biases)
    return tf.matmul(hidden, layer4_weights) + layer4_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.05).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [8]:
num_steps = 1001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 50 == 0):
      print('Minibatch loss at step %d: %f' % (step, l))
      print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 2.780356
Minibatch accuracy: 18.8%
Validation accuracy: 9.8%
Minibatch loss at step 50: 1.597346
Minibatch accuracy: 50.0%
Validation accuracy: 32.6%
Minibatch loss at step 100: 1.357188
Minibatch accuracy: 50.0%
Validation accuracy: 69.8%
Minibatch loss at step 150: 0.630099
Minibatch accuracy: 81.2%
Validation accuracy: 78.2%
Minibatch loss at step 200: 0.512798
Minibatch accuracy: 87.5%
Validation accuracy: 74.8%
Minibatch loss at step 250: 0.882204
Minibatch accuracy: 68.8%
Validation accuracy: 76.2%
Minibatch loss at step 300: 0.223894
Minibatch accuracy: 93.8%
Validation accuracy: 79.8%
Minibatch loss at step 350: 0.343979
Minibatch accuracy: 93.8%
Validation accuracy: 80.3%
Minibatch loss at step 400: 0.629292
Minibatch accuracy: 81.2%
Validation accuracy: 80.5%
Minibatch loss at step 450: 0.726870
Minibatch accuracy: 75.0%
Validation accuracy: 81.0%
Minibatch loss at step 500: 0.983654
Minibatch accuracy: 75.0%
Validation accuracy: 80.6%
Mi

---
Problem 2
---------

Try to get the best performance you can using a convolutional net. Look for example at the classic [LeNet5](http://yann.lecun.com/exdb/lenet/) architecture, adding Dropout, and/or adding learning rate decay.

---

In [5]:
batch_size = 50
kernel_size = 5
pooling_size = 2
channels_one = 6
channels_two = 16
hidden1_size = 120
hidden2_size = 84
dropout_keep_rate = 0.5
lam = 0.01


graph = tf.Graph()
with graph.as_default():
    #input data
    tf_train_dataset = tf.placeholder(tf.float32,shape=(batch_size, image_size, image_size, num_channels))
    tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
    
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    
    tf_test_dataset = tf.constant(test_dataset)
    tf_test_labels = tf.constant(test_labels)
    
    #variables
    layer_weight1 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, num_channels, channels_one],stddev=0.1))
    layer_biases1 = tf.Variable(tf.zeros([channels_one]))
    
    layer_weight3 = tf.Variable(tf.truncated_normal([kernel_size, kernel_size, channels_one, channels_two], stddev=0.1))
    layer_biases3 = tf.Variable(tf.constant(1.0, shape=[channels_two]))
    
    neuron_num = ((((image_size-pooling_size)//pooling_size+1-kernel_size+1-pooling_size)//pooling_size+1)**2)*channels_two
    layer_weight5 = tf.Variable(tf.truncated_normal([neuron_num, hidden1_size], stddev=0.1))
    layer_biases5 = tf.Variable(tf.constant(1.0, shape=[hidden1_size]))
    
    layer_weight6 = tf.Variable(tf.truncated_normal([hidden1_size, hidden2_size], stddev=0.1))
    layer_biases6 = tf.Variable(tf.constant(1.0, shape=[hidden2_size]))
    
    layer_weight7 = tf.Variable(tf.truncated_normal([hidden2_size, num_labels], stddev=0.1))
    layer_biases7 = tf.Variable(tf.constant(1.0, shape=[num_labels]))
    
    # Model
    def model(data):
        conv = tf.nn.conv2d(data, layer_weight1, [1,1,1,1], padding="SAME")+layer_biases1
        pooling = tf.nn.relu(tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding="VALID"))
        pooling = tf.nn.lrn(pooling, 4, bias=1.0,   alpha=0.001 / 9.0, beta=0.75, name='norm1')
        conv = tf.nn.conv2d(pooling, layer_weight3, [1,1,1,1], padding="VALID")+layer_biases3
        pooling = tf.nn.relu(tf.nn.max_pool(conv, [1,2,2,1], [1,2,2,1], padding="VALID"))
        pooling = tf.nn.lrn(pooling, 4, bias=1.0,   alpha=0.001 / 9.0, beta=0.75, name='norm2')
        shape = pooling.get_shape().as_list()
        reshape = tf.reshape(pooling,[shape[0],shape[1]*shape[2]*shape[3]])
        hidden1 = tf.nn.dropout(tf.matmul(reshape, layer_weight5)+layer_biases5, dropout_keep_rate)
        hidden2 = tf.nn.relu(tf.matmul(hidden1, layer_weight6)+layer_biases6)
        return tf.matmul(hidden2, layer_weight7)+layer_biases7
    
    #loss
    logits = model(tf_train_dataset)
    loss = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits))+lam*(tf.nn.l2_loss(layer_weight1)
             +tf.nn.l2_loss(layer_weight3)+
             tf.nn.l2_loss(layer_weight5)+tf.nn.l2_loss(layer_weight6)+tf.nn.l2_loss(layer_weight7))
    
    #optimizer
    global_step = tf.Variable(0)  # count the number of steps taken.
    learn_rate = tf.train.exponential_decay(0.05, global_step, 100, 0.9, staircase = True)
    optimizer = tf.train.GradientDescentOptimizer(learn_rate).minimize(loss)
    
    # Predictions for the training, validation, and test data.
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
    test_prediction = tf.nn.softmax(model(tf_test_dataset))

In [6]:
num_steps = 10001

with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    print("initialed")
    for step in range(num_steps):
        index = np.random.randint(0, train_labels.shape[0], batch_size)
        batch_data = train_dataset[index, :, :, :]
        batch_labels = train_labels[index, :]
        _, l, predictions = sess.run(
            [optimizer, loss, train_prediction], feed_dict={tf_train_dataset:batch_data, tf_train_labels:batch_labels})
        if (step % 1000 == 0):
          print('Minibatch loss at step %d: %f' % (step, l))
          print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
          print('Validation accuracy: %.1f%%' % accuracy(valid_prediction.eval(), valid_labels))
    print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))
    

initialed
Minibatch loss at step 0: 5.860135
Minibatch accuracy: 10.0%
Validation accuracy: 10.3%
Minibatch loss at step 1000: 1.577939
Minibatch accuracy: 82.0%
Validation accuracy: 80.7%
Minibatch loss at step 2000: 1.003533
Minibatch accuracy: 84.0%
Validation accuracy: 83.8%
Minibatch loss at step 3000: 0.791233
Minibatch accuracy: 86.0%
Validation accuracy: 84.5%
Minibatch loss at step 4000: 0.705345
Minibatch accuracy: 86.0%
Validation accuracy: 84.5%
Minibatch loss at step 5000: 0.641133
Minibatch accuracy: 88.0%
Validation accuracy: 84.6%
Minibatch loss at step 6000: 1.013062
Minibatch accuracy: 82.0%
Validation accuracy: 85.0%
Minibatch loss at step 7000: 0.765789
Minibatch accuracy: 88.0%
Validation accuracy: 84.2%
Minibatch loss at step 8000: 0.762589
Minibatch accuracy: 80.0%
Validation accuracy: 85.0%
Minibatch loss at step 9000: 0.809168
Minibatch accuracy: 76.0%
Validation accuracy: 84.0%
Minibatch loss at step 10000: 0.655323
Minibatch accuracy: 86.0%
Validation accurac